# Weapon target assignment

Versione WTA con **ricerca esaustiva**

---
Per commenti, modifiche et al. contattateci via email:
- marco.vannucci@santannapisa.it
- valentina.colla@santannapisa.it

## Importazioni e variabili globali

In [1]:
import numpy as np
hc=0 ## questa è una variabile globale

## Funzioni accessorie

### Calcolo possibili combinazioni *weapon-target*

La funzione `combina` calcola tutte le possibili combinazioni di `N_weapon` armi su `N_target` bersagli e le mette nella matrice `all_mat`, le cui dimensioni sono note e precedentemente calcolate, essendo le combinazioni `N_target^N_weapon`

In [2]:
def combina(lista_b, N_weapon, N_target, out_mat, out_pos, all_mat):
    global hc
    for k in range(N_target):
        out_mat[out_pos]=lista_b[k]
        if N_weapon>1:
            combina(lista_b,N_weapon-1,N_target,out_mat,out_pos+1, all_mat)
        else:
            #print(out_mat) serve a visualizzare tutte le possibili combinazioni armi-bersaglio
            all_mat[hc]=out_mat
            hc=hc+1
    return

### Funzione obiettivo

La funzione `valuta_comb` calcola la funzione obiettivo da **minimizzare** per la combinazione armi-bersagli, contenuta nel vettore `combin`, che è il primo argomento.

La funzione obiettivo è la somma dei prodotti del valore di ciascun bersaglio moltiplicato per la probabilità  che esso sopravviva alle armi che sono a lui assegnate, che è *1 - la probabilità che l'arma colpisca il bersaglio*. 

La matrice delle probabilità che ciascun tipo di arma ha di colpire il bersaglio è uno degli argomenti, ossia `Mat_prob`. 
Fra gli argomenti passo anche il vettore `vett_W` che contiene il tipo di ciascuna delle armi della combinazione.
la funzione di compone di due cicli annidati.

**I cicli:**
- Il ciclo esterno itera sui bersagli e quindi calcola i singoli addendi della funzione obiettivo.
- Il ciclo interno itera sulle armi e serve a calcolare la probabailità che ciascun bersaglio sopravvivva all'attacco, che è una produttoria delle probabilità che ha di sopravvivere a ciascuna delle armi. Se un'arma è assegnata proprio a lui, il fattore della produttoria è *(1-p)*, se invece non lo è, il fattore della produttoria è *1* e  quindi, in pratica, non si fa alcuna moltiplicazione.


In [3]:
def valuta_comb(combin, vett_W, Mat_prob, valori_T):
    valore=0
    n_T=len(valori_T)
    n_W=len(combin)
    for i in range(n_T): # il ciclo esterno calcola ogni addendo della funzione obiettivo da minimizzare, relativo a ciascun target
        prob_sopravv_target=1;
        for j in range(n_W):
            if combin[j]==int(i+1): # Il ciclo interno calcola la probabilità  di sopravvivenza di ciascun target 
                prob_sopravv_target*=(1-Mat_prob[vett_W[j]-1,i])
        valore+=valori_T[i]*prob_sopravv_target
    return valore

## Main

Parametri specificati dall'utente:
- il numero dei bersagli
- il numero di tipi di armi.

Parametri settati automaticamente:
- il valore dei bersagli è un valore intero casuale tra 0 e 100
- la probabilità che un tipo di arma colpisca ciascun bersaglio è anche essa casuale

**Codifica soluzione**

Ogni combinazione armi-bersagli è codificata come una *stringa di interi* di lunghezza pari al numero totale di armi disponibili, **nell'ordine in cui i tipi vengono forniti**.

> **Ad esempio**: se io ho 2 armi di tipo A e 3 armi di tipo B, la combinazione è lunga 5 e così composta [A1, A2, B1, B2, B3]

Il contenuto di ciascun elemento è il numero del target a cui l'arma corrispondente è assegnata.
Chiaramente, armi dello stesso tipo hanno la medesima probabilità  di colpire ogni bersaglio.
La matrice delle probabilità  che ciascun tipo di arma ha di colpire ogni bersaglio si chiama `Mat_prob_cogliere_bersagli` ed ha tante righe quante sono i tipi di armi e tante colonne quante sono i bersagli.

> Ho creato anche una lista che ha tante righe quane i tipi di armi, che è una lista mista, che contiene per ogni riga il nome del tipo di arma, il numero di armi di quel tipo e il vettore delle probabilità  che questo tipo di arma colpisca i diversi bersagli.
Questa lista non serve assolutamente a niente, l'ho creata per sfizio e come struttura dati che raccoglie tutte le informazioni relative alle ami. 

In [10]:
n_bersagli=int(input('Quanti bersagli ci sono? ') )
#lista_bersagli = [1, 2, 3]
lista_bersagli=list(range(1,n_bersagli+1))

# determino casualmente il valore di ciascun bersaglio
Valori_bersagli = np.array(100*np.random.rand(n_bersagli),dtype=int)

print('\n%12s   %8s'%('Bersaglio','Valore'))
for i in lista_bersagli:
    print('%12d   %8d'%(i,Valori_bersagli[i-1]))

Quanti bersagli ci sono?  5



   Bersaglio     Valore
           1         22
           2          2
           3         35
           4         83
           5         51


In [11]:
# inserimento tipologia e delle armi
n_tipo_armi=int(input('Quanti tipi di armi ci sono? ') )

# numero di armi per ciascun tipo 
cont_armi_per_tipo=np.zeros(n_tipo_armi, dtype=int)

# matrice con la probabilità, per ciascuna tipologia di arma, di colpire ciascun bersaglio
Mat_prob_cogliere_bersagli=np.zeros([n_tipo_armi,n_bersagli])

# descrittore del set di armi (include nome, tipo e le probabilità di colpire i bersagli)
lista_armi=[] 


for j in range(n_tipo_armi):
    nome = 'WEATYPE_%d'%j
    probab_cogliere_bersagli=np.random.rand(n_bersagli)
    Mat_prob_cogliere_bersagli[j]=probab_cogliere_bersagli
    testo ='Quante armi ci sono del tipo '+ nome + '? '
    cont_armi_per_tipo[j]=input(testo)
    lista_armi.append([nome, cont_armi_per_tipo[j],probab_cogliere_bersagli])

    
#for l in lista_armi:
#    print(l)

Quanti tipi di armi ci sono?  2
Quante armi ci sono del tipo WEATYPE_0?  3
Quante armi ci sono del tipo WEATYPE_1?  3


### Simulazione delle combinazioni

In [12]:
W_tot=int(np.sum(cont_armi_per_tipo)) #numero totale delle armi
Vec_tipo_armi=np.zeros(W_tot,dtype=int) #vettore con il tipo di ciascuna arma
offset=0
for i in range(n_tipo_armi):
    for j in range(cont_armi_per_tipo[i]):
        Vec_tipo_armi[j+offset]=i+1;
    offset+=cont_armi_per_tipo[i]
    
# adesso faccio decidere all'utente se andare avanti con il calcolo in base al numero effettivo delle combinazioni da valutare
N_comb=int(n_bersagli**W_tot) #numero totale delle combinazioni armi-bersagli

print('Saranno valutate %d combinazioni arma-target'%N_comb)
cont=input('Continuare? [s/n]')

print(cont)

Saranno valutate 15625 combinazioni arma-target


Continuare? [s/n] s


s


In [13]:
if cont=='s':
    Tutte_combin=np.zeros([N_comb, W_tot], dtype=int) #array che contiene tutte le associazioni possibili delle armi ai bersagli
    assoc=np.zeros(W_tot)
    out_pos=0
    combina(lista_bersagli, W_tot, n_bersagli, assoc, out_pos, Tutte_combin)

    Valore_combin=np.zeros(N_comb)
    for i in range(N_comb):
        Valore_combin[i]=valuta_comb(Tutte_combin[i],Vec_tipo_armi,Mat_prob_cogliere_bersagli,Valori_bersagli)
    indx_Best=np.argmin(Valore_combin)
    Best_combin=Tutte_combin[indx_Best]
    Min_obj=min(Valore_combin)
    print("la migliore combinazione è ", Best_combin)
    print("Il valore della funzione obiettivo in ", Best_combin, " è ", Min_obj)

la migliore combinazione è  [4 5 5 1 3 5]
Il valore della funzione obiettivo in  [4 5 5 1 3 5]  è  18.541816439032353
